In [144]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from dateutil import parser
from datetime import datetime
from crontab import CronTab

In [93]:
# scrapes Nova Scotia data
def web_scrape_NS(soup):
    data = []
    table = soup.find_all('td', attrs={'width': '45%'})
    table = [ele.text.strip() for ele in table] # strip tags
    data.append([ele for ele in table if ele]) # strip empty values
    data = data[0]

    load = findValue('Net Load', data)
    wind = findValue('Wind Generation', data)
    return load, wind

In [94]:
# scrapes New Brunswick data
def web_scrape_NB(soup):
    load_box = soup.find('td', attrs={'id': 'nb-load'})
    load = load_box.text.strip() # strip() is used to remove starting and trailing
    return load

In [116]:
# scrapes Ontario data
def web_scrape_ON(soup):
    span = soup.findAll(string=re.compile("MW")) #'div', class_="col-sm-3")
    for i in range(len(span)):
        span[i] = span[i][:len(span[i])-3]
    load = span[1]
    types = [span[12], span[13], span[14], span[15], span[16], span[17]]
    return load, types

In [96]:
# scrapes Newfoundland and Labrador data
def web_scrape_NF(soup):
    loadStr = soup.find_all(string=re.compile("MW"))[0]
    load = loadStr[:len(loadStr)-4]
    return load

In [97]:
# scrapes Alberta data
def web_scrape_AB(soup):
    link = soup.find(string = re.compile("Alberta Total Net Generation"))
    load = link.find_next(string = True)
    srcs = ["COAL", "GAS", "HYDRO", "OTHER", "WIND"]
    sources = []
    for i in range(len(srcs)):
        sources.append(soup.find(string = re.compile(srcs[i])).find_next(string = True).find_next(string = True))
    return load, sources

In [98]:
def findValue(element, data):
    i = 0
    for ele in data:
        if ele == element:
            loadIndex = i
        else:
            i += 1
    load = data[loadIndex+1]
    return load

# returns the BeautifulSoup given a url to scrape
def urlOpen(url):
    html = urllib.request.urlopen(url)
    return BeautifulSoup(html, 'html.parser')

In [145]:
urls = {"NF": "https://nlhydro.com/system-information/supply-and-demand/", 
        "ON": "http://www.ieso.ca/en/Power-Data/This-Hours-Data", 
        "NB": "https://tso.nbpower.com/Public/en/SystemInformation_realtime.asp",
        "NS": "https://resourcesprd-nspower.aws.silvertech.net/oasis/current_report.shtml",
        "AB": "http://ets.aeso.ca/ets_web/ip/Market/Reports/CSDReportServlet"}
soupNF = urlOpen(urls["NF"])
soupON = urlOpen(urls["ON"])
soupNB = urlOpen(urls["NB"])
soupNS = urlOpen(urls["NS"])
soupAB = urlOpen(urls["AB"])
time = str(datetime.now().hour) + ":" + str(datetime.now().minute)

loadList = [loadNF, loadON, loadNB, loadNS, loadAB]
# strip any commas from the load values
for i in range(len(loadList)):
    loadList[i] = loadList[i].replace(',', '')
windNS = windNS.replace(',', '')
for i in range(len(sourcesON)):
    sourcesON[i] = sourcesON[i].replace(',', '')
for i in range(len(sourcesAB)):
    sourcesAB[i] = sourcesAB[i].replace(',', '') 

    
provinces = ["Newfoundland and Labrador", "Ontario", "New Brunswick", "Nova Scotia", "Alberta"]
dfLoad = pd.DataFrame(columns=['Time','Net Load (MW)', 'Province'])
for i in range(len(timeList)):
    dfLoad = dfLoad.append({'Time': time, 'Net Load (MW)': loadList[i], 'Province': provinces[i]}, ignore_index = True)
dfLoad.to_csv('Energy.csv', index = False, encoding='cp1252')